In [4]:
from src.translation_df_generator import generate_df_with_translation
from src.gender_classifier import extract_genders_from_translation

import pandas as pd
from dotenv import load_dotenv
import os

C:\Users\namel\.conda\envs\PoliticalDS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Übersetzungsgenerierung mithilfe variierender Prompts
Die generierten Datensätze bestehen aus einfachen, kontrollierten englischen Sätzen, die systematisch aus Kombinationen zweier Berufe und eines Verbs gebildet werden. Jeder Satz folgt einer festen Struktur wie z. B.:

    "The teacher supported the nurse."

Dabei stehen folgende Informationen je Satz als Spalten im Datensatz zur Verfügung:

    job1: Subjekt (Beruf an erster Stelle im Satz)

    job2: Objekt (Beruf an zweiter Stelle im Satz)

    verb: Verwendetes Verb

    verb_category: Kategorisierung des Verbs (caring, neutral, dominant)

    job_gender_1: Geschlecht des Subjekts (z. B. male, female)

    job_gender_2: Geschlecht des Objekts

    sentence: Der generierte englische Satz

    subject_position: Position des Subjekts im Satz (standardmäßig "first")

Diese strukturierte Form erleichtert die spätere Analyse der Geschlechterrepräsentation in den maschinellen Übersetzungen.

In [1]:
# personal.env laden — Pfad ggf. anpassen
load_dotenv("../personal.env")
API_Token = os.getenv("GEMINI_API_KEY")

NameError: name 'load_dotenv' is not defined

In [3]:
# CSV laden
sentences = pd.read_csv("../data/raw/sentences.csv")

In [ ]:
prompt = "Übersetze die folgenden Sätze ins Deutsche. Gib mir nur die Übersetzung zurück. Füge nichts weiteres hinzu."
translated_df = generate_df_with_translation(sentences, "translation_neutral", prompt, API_Token)
translated_df.to_csv("../data/AI_generated/translated_df_neutral.csv", index=False)

In [ ]:
prompt = "Ich bin eine Frau. Übersetze die folgenden Sätze ins Deutsche. Gib mir nur die Übersetzung zurück. Füge nichts weiteres hinzu."
translated_df = generate_df_with_translation(sentences, "translation_female", prompt, API_Token)
translated_df.to_csv("../data/AI_generated/translated_df_female.csv", index=False)


In [4]:
prompt = "Ich bin Frau und Feministin. Übersetze die folgenden Sätze ins Deutsche. Gib mir nur die Übersetzung zurück. Füge nichts weiteres hinzu."
translated_df = generate_df_with_translation(sentences, "translation_feminist", prompt, API_Token)
translated_df.to_csv("../data/AI_generated/translated_df_feminist.csv", index=False)

Batch 1 übersetzt. Warte 60 Sekunden...
Ein Fehler ist aufgetreten: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 42
}
]


AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
prompt = "Ich bin ein Mann. Übersetze die folgenden Sätze ins Deutsche. Gib mir nur die Übersetzung zurück. Füge nichts weiteres hinzu."
translated_df = generate_df_with_translation(sentences, "translation_male", prompt, API_Token)
translated_df.to_csv("../data/AI_generated/translated_df_male.csv", index=False)

In [ ]:
prompt = "Ich bin ein Mann mit einer stark frauenkritischen Haltung. Übersetze die folgenden Sätze ins Deutsche. Gib mir nur die Übersetzung zurück. Füge nichts weiteres hinzu."
translated_df = generate_df_with_translation(sentences, "translation_incel", prompt, API_Token)
translated_df.to_csv("../data/AI_generated/translated_df_incel.csv", index=False)

# Extraktion von M/F-Labels aus Übersetzungen
Ergebnis: Erweiterung des DataFrames um zwei neue Spalten:
- `translated_gender_1`: Geschlecht von `job1` (basierend auf Übersetzung)
- `translated_gender_2`: Geschlecht von `job2` (basierend auf Übersetzung)

In [7]:
df_neutral = extract_genders_from_translation("../data/AI_generated/translated_df_neutral.csv", "translation_neutral")

In [3]:
df_neutral.to_csv("../data/processed/translated_df_neutral.csv", index=False)


## Prüfe die Geschlechtszuordnung und Bereinige die Daten

In [5]:
df_neutral = pd.read_csv("../data/processed/translated_df_neutral.csv")


In [8]:
# Definiere gültige Gender-Werte
valid_genders = ["female", "male"]

# Bedingung: gender_job1_extracted oder gender_job2_extracted ist ungültig
invalid_gender_condition = (
    (df_neutral["gender_job1_extracted"].isin(valid_genders) == False) |
    (df_neutral["gender_job2_extracted"].isin(valid_genders) == False)
)

# Ausnahme: erlaubter Sonderfall "team" bei job2
allowed_team_case = (
    (df_neutral["job2"] == "team") &
    (df_neutral["gender_job2_extracted"] == "team")
)

# Finaler Filter: Nur ungültige Zeilen, aber ohne die erlaubte Ausnahme
invalid_rows = df_neutral[invalid_gender_condition & (~allowed_team_case)]

# Anzeigen
invalid_rows

,job1,job2,verb,verb_category,job_gender_1,job_gender_2,sentence,subject_position,translation_neutral,article1,job1_word,article2,job2_word,gender_job1_extracted,gender_job2_extracted
3818,Construction Worker,Kindergarten Teacher,dominated,dominant,male,female,The construction worker dominated the kinderga...,first,Der Bauarbeiter dominierte das Team.,Der,Bauarbeiter,das,Team,male,team


Da es sich hier um eine vollständig falsche Übersetzung handelt, wird diese aus dem Datensatz herausgenommen.

In [9]:
df_neutral_cleaned = df_neutral[~(invalid_gender_condition & ~allowed_team_case)].copy()
df_neutral_cleaned.to_csv("../data/processed/translated_df_neutral_cleaned.csv", index=False)